In [1]:
from io import StringIO
import datetime
import boto3
import json
import time
import os

from zipfile import ZipFile

s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
lambda_client = boto3.client('lambda')

import pickle,copy,os,re,datetime,io
import pandas as pd
import numpy as np
from collections import Counter, deque, defaultdict
from datetime import timedelta as td
from datetime import datetime as dt

import matplotlib.pyplot as plt
import matplotlib as mpl

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor

from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,OrdinalEncoder,LabelEncoder

import pickle

pd.options.mode.chained_assignment = None

In [46]:
PICKLE_DIR = os.getcwd()+'/pickle/' #os.path.dirname(os.path.realpath(__file__))+'/pickle/'

#####
##### **HELPER FUNCTIONS**

In [2]:
def pickly(t, f, v=None):
    m = 'wb' if t == 'dump' else 'rb'
    with open(f, m) as file:
        if t == 'dump':
            pickle.dump(v, file)
        elif t == 'load':
            return pickle.load(file)
        else:
            pass


def make_int(c):
    try:
        return int(c) # 0 if np.isnan(int(c)) else int(c)
    except:
        return 0


def day_diff(d1,d2,t):
    try:
        return int((d1-d2).days)
    except:
        return np.nan


def delcols(df,c):
    try:
        del df[c]
    except:
        pass


def datefill(d,end=None,numdays=0):
    if end is None:
        end = max(d) #.date()
    dr = (end-min(d)).days #.date()
    dl = [end - datetime.timedelta(days=x) for x in range(dr+1+numdays)]
    # d = [d.date() for d in d]
    date_list = [a for a in dl if a not in d]
    if numdays > 0: return date_list[:numdays]
    return date_list


def get_counts_list(col, name, ascending=False, lim=None):
    data = pd.DataFrame.from_dict([dict(Counter(col))]).T.sort_values(by=0, ascending=ascending).reset_index()
    if lim is not None:
        data = data[:lim]
    data.columns = [name, 'count']
    return data


def get_dupes(col):
    df = pd.DataFrame.from_dict([dict(Counter(col))]).T.sort_values(by=0,ascending=False).reset_index()
    dupes = df[df[0]>1]
    if len(dupes)>0: return dupes
    print('No Dupes')


# team lookup to address the long-tail and reconcile franchises
def get_team_names(games, output='dict', default='Other'):
    team_replacements = {'NOH': 'NOP', 'NOK': 'NOP', 'SAN': 'SAS', 'GOS': 'GSW', 'UTH': 'UTA', 'PHL': 'PHI'}
    active_teams = get_counts_list(games[games['away'] != 'No Games']['home'], 'team_raw', lim=40) # these are the valid teams
    active_teams['team'] = active_teams['team_raw'].apply(lambda x: team_replacements[x] if x in team_replacements.keys() else x)
    if output=='dict':
        def def_value():
            return default
        team_dict = defaultdict(def_value)
        for i in range(len(active_teams)):
            team_dict[active_teams['team_raw'][i]] = active_teams['team'][i]
        return team_dict
    return active_teams[['team_raw', 'team']]


def etl_process(data, tasks):
    for t in tasks:
        data = t(data)
    return data


def get_conference(team=None):
    confs = {'East': ['BOS', 'MIL', 'PHI', 'CLE', 'BKN', 'MIA', 'NYK', 'ATL', 'WAS', 'CHI', 'TOR', 'IND', 'ORL', 'DET', 'CHA'],
             'West': ['DEN', 'MEM', 'SAC', 'PHX', 'DAL', 'LAC', 'NOP', 'MIN', 'GSW', 'OKC', 'UTA', 'POR', 'LAL', 'SAS', 'HOU']}
    if team is not None:
        return 'East' if team in confs['East'] else 'West'
    return confs


def bucket_data(bucket_name):
    return pd.DataFrame(boto3.client('s3').list_objects(Bucket=bucket_name)['Contents'])


def fetch_files():
    games = read_s3('data/games.csv', output='dataframe')
    boxes = read_s3('data/boxes.csv', output='dataframe')
    plays = read_s3('data/plays.csv', output='dataframe')
    return games, boxes, plays

def read_csv_from_zip(zip_path):
    zip_file = ZipFile(zip_path)
    file_name = zip_path.split('/')[-1].split('.')[0]+'.csv'
    print(file_name)
    for z in zip_file.infolist():
        if z.filename == file_name:
            return pd.read_csv(zip_file.open(file_name))

#####
##### **TRIGGER LAMBDA (IF NOT SCHEDULED) AND VERIFY UPDATES**

In [3]:
lambda_payload_1 = {
    'table': 'games',
    'num_pages': 'infer',
    # 'dates':['2023-02-23','2023-02-24'],
    'batch_size': 20,
    'rand': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    'include_plays': 'true',
    'allow_repeats': 'true'
}

lambda_client.invoke(
    FunctionName='selenium-trigger-1',
    InvocationType='Event',
    Payload=json.dumps(lambda_payload_1)
);

In [5]:
# retrieve files from lambda, once they are ready
# we check every 15 seconds for their availability

timeout = 900
update_threshold = 900
delay = 30
time_elapsed = 0
files = ['data/games.zip', 'data/boxes.zip']
utc_time = datetime.datetime.now(datetime.timezone.utc)

while time_elapsed < timeout:
    my_bucket_data = bucket_data('hwm-nba')
    updated_utc = min(my_bucket_data[my_bucket_data['Key'].isin(files)]['LastModified']).to_pydatetime()
    timenow_utc = datetime.datetime.now(datetime.timezone.utc)
    diff = (timenow_utc - updated_utc)
    if (diff.days*86400)+(diff.seconds) <= update_threshold:
        print('updated files found!')
        # games, boxes, plays = fetch_files()
        timefetch_utc = datetime.datetime.now(datetime.timezone.utc)
        time_taken = (timefetch_utc-utc_time).seconds
        print('retrieved 3 updated files from S3 after', time_taken, 'seconds\n')
        time_elapsed = timeout
    else:
        time_elapsed += delay
        if time_elapsed >= timeout: 
            print(timeout,'seconds elapsed - timing out\n')
        else:
            print('files not ready - reverifying file updates')
        time.sleep(delay)

files not ready - reverifying file updates
files not ready - reverifying file updates


KeyboardInterrupt: 

#####
##### **LOAD DATA**

In [6]:
games = pd.read_csv('../data/games.csv')
# boxes = pd.read_csv('../data/boxes.csv')
# plays = pd.read_csv('../data/plays.csv')

In [7]:
games

,Date,Away,AS,Home,HS,OT,Detail Path,Game Type,Detail Data,Rand
0,1995-02-16,CLE,106,MIL,85,N,cle-vs-mil-0029400649,regular,1,8
1,1995-02-16,DEN,94,NJN,100,N,den-vs-njn-0029400651,regular,1,9
2,1995-02-16,HOU,105,CHH,89,N,hou-vs-chh-0029400652,regular,1,9
3,1995-02-16,LAL,82,SAC,98,N,lal-vs-sac-0029400648,regular,1,8
4,1995-02-16,NYK,96,MIA,87,N,nyk-vs-mia-0029400650,regular,1,4
...,...,...,...,...,...,...,...,...,...,...
42246,2023-03-12,CLE,-,CHA,-,N,cle-vs-cha-0022201015,regular,0,6
42247,2023-03-12,NYK,-,LAL,-,N,nyk-vs-lal-0022201019,regular,0,5
42248,2023-03-12,OKC,-,SAS,-,N,okc-vs-sas-0022201018,regular,0,6
42249,2023-03-12,POR,-,NOP,-,N,por-vs-nop-0022201017,regular,0,3


In [8]:
# creation of game features
# boxes.columns = [re.sub(' ','_',c.lower()) for c in list(boxes.columns)]

In [9]:
games.columns = [re.sub(' ','_',c.lower()) for c in list(games.columns)]
team_names_dict = get_team_names(games)

#####
##### **BASIC FEATURES**

In [10]:
def basic_features(games):
    filters = []
    filters.append("away != 'No Games'")
    filters.append("away != '0'")
    filters.append("hs >= 50")
    filters.append("hs != '-'")
    filters.append("diff != 0")
    filters.append("date >= '1996'")

    # games.columns = [re.sub(' ','_',c.lower()) for c in list(games.columns)]
    games = games.drop_duplicates(subset=['detail_path'], keep='last')
    
    games.loc[:,'hs'] = games.loc[:,'hs'].apply(lambda x: make_int(x))
    games.loc[:,'as'] = games.loc[:,'as'].apply(lambda x: make_int(x))

    # clean up home and away scores
    games['ot'] = games.apply(lambda x: 0 if (x['ot']=='N' or x['away']=='No Games') \
                              else (1 if int(max(x['as'],x['hs']))<200 else int(str(max(x['as'],x['hs']))[:1])),axis=1)

    games['as'] = games.apply(lambda x: x['as'] if (x['ot']<2 or x['as']<200) else x['as'] % 1000, axis=1)
    games['hs'] = games.apply(lambda x: x['hs'] if (x['ot']<2 or x['hs']<200) else x['hs'] % 1000, axis=1)

    games['game_subtype'] = games['game_type'].apply(lambda x: re.sub('^(east|west)-', '', re.sub('-[0-9]{1}$','',x)))
    games['game_type'] = games['game_type'].apply(lambda x: x if x in ['preseason','regular'] else 'playoffs')

    games['diff'] = games.apply(lambda r: np.abs(r['hs']-r['as']),axis=1)
    games1 = games.query(' & '.join(filters)).reset_index(drop=True)
    games2 = games[(games['date'] >= dt.strftime(dt.today(),'%Y-%m-%d')) & (games['away'] != 'No Games')].reset_index(drop=True)
    
    games = pd.concat([games1, games2]).reset_index(drop=True)
    
    # clean the team names - reconcile old franchises with new, and aggregate the long tail of guest/novelty teams
    games['home'] = games['home'].apply(lambda x: team_names_dict[x])
    games['away'] = games['away'].apply(lambda x: team_names_dict[x])
        
    games['month'] = games['date'].apply(lambda x: x[5:7])
    games['date'] = games['date'].apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
    games['winner'] = games.apply(lambda x: x['home'] if x['hs'] >= x['as'] else x['away'], axis=1)
    games['home_win'] = games.apply(lambda x: 1 if x['hs'] >= x['as'] else 0, axis=1)
    games['season'] = games['date'].apply(lambda x: str((x-td(days=225)).year)+'-'+str((x-td(days=225)).year+1)) # mid-august split
    # exceptions
    games['season'] = games.apply(lambda x: '2019-2020' if (dt.strftime(x['date'],'%Y-%m-%d')>='2020-03-01' \
                                                            and dt.strftime(x['date'],'%Y-%m-%d')<='2020-11-15') else x['season'], axis=1)

    games['is_preseason'] = games.apply(lambda x: 1 if x['game_type']=='preseason' else 0,axis=1)
    games['is_playoffs'] = games.apply(lambda x: 0 if x['game_type'] in ['preseason','regular'] else 1,axis=1)
    
    games['team_pair'] = games.apply(lambda r: [r['away'], r['home']], axis=1)
    games['team_pair'].apply(lambda r: r.sort())
    games['team_pair'] = games['team_pair'].apply(lambda r: '-'.join(r))
    
    del games['detail_data']
    del games['rand']
    
    return games

#####
##### **PLAYOFF FEATURES**

In [11]:
def playoff_features(games): # etl the playoff calculations
    games.columns = [re.sub(' ','_',c.lower()) for c in list(games.columns)]
    playoff_games = games[games['game_type']=='playoffs'].iloc[:,:].sort_values(by=['season', 'team_pair', 'date']).reset_index(drop=True)

    playoff_games['winner_1'] = playoff_games.apply(lambda r: 1 if r['winner']==r['team_pair'][:3] else 0, axis=1)
    playoff_games['winner_2'] = playoff_games.apply(lambda r: 1 if r['winner']==r['team_pair'][-3:] else 0, axis=1)

    playoff_games[['t1_wins_after_game', 't2_wins_after_game']] = \
        playoff_games[['season', 'team_pair', 'winner_1', 'winner_2']].groupby(['season', 'team_pair'])[['winner_1', 'winner_2']].transform(pd.Series.cumsum)
    del playoff_games['winner_1']
    del playoff_games['winner_2']

    playoff_games['leader_after_game'] = playoff_games.apply(lambda r: r['team_pair'][:3] if r['t1_wins_after_game']>r['t2_wins_after_game'] \
                                                             else (r['team_pair'][-3:] if r['t2_wins_after_game']>r['t1_wins_after_game'] \
                                                                   else 'tied series'), axis=1)
    playoff_games[['season2', 'team_pair2', 't1_wins_before_game', 't2_wins_before_game', 
                   'leader_before_game']] = playoff_games[['season', 'team_pair', 't1_wins_after_game',
                                                           't2_wins_after_game', 'leader_after_game']].shift(periods=1)
    
    playoff_games['t1_wins_before_game'] = playoff_games.apply(lambda r: 0 if np.isnan(r['t1_wins_before_game']) \
                                                               or r['season']!=r['season2'] else int(r['t1_wins_before_game']), axis=1)
    playoff_games['t2_wins_before_game'] = playoff_games.apply(lambda r: 0 if np.isnan(r['t2_wins_before_game']) \
                                                               or r['season']!=r['season2'] else int(r['t2_wins_before_game']), axis=1)
    playoff_games['leader_before_game'] = playoff_games.apply(lambda r: 'series starting' \
                                                              if str(r['leader_before_game'])=='nan' or r['season']!=r['season2'] \
                                                              else r['leader_before_game'], axis=1)
    del playoff_games['season2']
    del playoff_games['team_pair2']
    
    playoff_series_winners = playoff_games.drop_duplicates(subset=['season', 'team_pair'],keep='last').reset_index(drop=True)
    playoff_series_winners = playoff_series_winners[['date', 'season', 'game_type', 'team_pair', 'winner']]
    playoff_series_winners = playoff_series_winners.sort_values(by=['winner', 'date']).reset_index(drop=True)
    playoff_series_winners['count'] = 1

    playoff_series_winners['playoff_round'] = playoff_series_winners[['winner', 'season', 'game_type',
                                                                      'count']].groupby(['winner', 'season', 'game_type'])['count'].transform(pd.Series.cumsum)
    del playoff_series_winners['count']

    playoff_series_winners = playoff_series_winners[['winner', 'season', 'game_type', 'team_pair', 'playoff_round']]
    playoff_series_winners.columns = ['series_winner', 'season', 'game_type', 'team_pair', 'playoff_round']

    playoff_games = playoff_games.merge(playoff_series_winners, how='left', on=['season', 'team_pair', 'game_type'], sort=False)
    
    playoff_games['knockout_game'] = playoff_games.apply(lambda r: 1 if max(r['t1_wins_before_game'], r['t2_wins_before_game'])==3 \
                                                         else (1 if max(r['t1_wins_before_game'], r['t2_wins_before_game'])==2 and \
                                                               r['playoff_round']==1 and r['season'] < '2002' else 0), axis=1)
    
    playoff_games = playoff_games[['date', 'detail_path', 't1_wins_after_game', 't2_wins_after_game', 'leader_after_game', 't1_wins_before_game', 't2_wins_before_game',
                                   'leader_before_game', 'series_winner', 'playoff_round', 'knockout_game']]
    
    games = games.merge(playoff_games, how='left', on=['date', 'detail_path'], sort=False)
    
    games['game_subtype'] = games['playoff_round'].apply(lambda x: 0 if np.isnan(x) else int(x))
    del games['playoff_round']

    return games

#####
##### **STREAKS WON AND LOST (deprecated)**

In [12]:
# def streaks_won_lost(games):
#     games.columns = [re.sub(' ','_',c.lower()) for c in list(games.columns)]
#     days_played_away = games[games['as']>0][['date', 'detail_path', 'away', 'winner', 'season', 'is_preseason', 'is_playoffs', 'diff']]
#     days_played_away['type'] = 'away'
#     days_played_home = games[games['hs']>0][['date', 'detail_path', 'home', 'winner', 'season', 'is_preseason', 'is_playoffs', 'diff']]
#     days_played_home['type'] = 'home'
#     days_played_away.columns = days_played_home.columns = ['date', 'detail_path', 'team', 'winner', 'season', 'is_preseason', 'is_playoffs', 'diff', 'type']
#     days_played = days_played_away.append(days_played_home).sort_values(by=['team', 'date'],ascending=[True,True]).drop_duplicates()
#     days_played['won'] = days_played.apply(lambda x: 1 if x['winner'] == x['team'] else 0, axis=1)
#     days_played['lost'] = 1-days_played['won']
#     del days_played['winner']
    
#     dummy_games = days_played.drop_duplicates(subset=['team'], keep='first')
#     dummy_games['date'] = dummy_games['date'].apply(lambda x: dt.strptime('1900-01-01', '%Y-%m-%d').date())
#     dummy_games['won'] = dummy_games['lost']
#     dummy_games['lost'] = 1-dummy_games['won']
    
#     days_played = days_played.append(dummy_games).sort_values(by=['team','date'],ascending=[True,True]).reset_index(drop=True).reset_index()
    
#     days_played['streak_map_won'] = days_played.groupby('team')['lost'].transform(pd.Series.cumsum)
#     days_played['streak_map_lost'] = days_played.groupby('team')['won'].transform(pd.Series.cumsum)

#     streak_map_won  = days_played[['index', 'team', 'streak_map_won']].drop_duplicates(subset=['team','streak_map_won'],keep='first')
#     streak_map_lost = days_played[['index', 'team', 'streak_map_lost']].drop_duplicates(subset=['team','streak_map_lost'],keep='first')

#     days_played = days_played.merge(streak_map_won,how='left',on=['team','streak_map_won'],sort=False)
#     days_played['streak_won'] = days_played['index_y'].shift(1)
#     days_played['team1'] = days_played['team'].shift(1)
#     days_played['streak_won'] = days_played.apply(lambda r: 0 if r['team']!=r['team1'] else int(r['index_x']-r['streak_won']-1),axis=1)
    
#     days_played = days_played.merge(streak_map_lost,how='left',on=['team','streak_map_lost'],sort=False)
#     days_played['streak_lost'] = days_played['index'].shift(1)
#     days_played['streak_lost'] = days_played.apply(lambda r: 0 if r['team']!=r['team1'] else int(r['index_x']-r['streak_lost']-1),axis=1)
    
#     days_played = days_played[days_played['date']>datetime.date(1900,1,1)].reset_index(drop=True)
#     days_played_home = days_played[days_played['type']=='home'][['date','detail_path','won','lost','streak_won','streak_lost']]
#     days_played_home.columns = ['date','detail_path','won_home','lost_home','streak_won_home','streak_lost_home']
#     days_played_away = days_played[days_played['type']=='away'][['date','detail_path','won','lost','streak_won','streak_lost']]
#     days_played_away.columns = ['date','detail_path','won_away','lost_away','streak_won_away','streak_lost_away']
    
#     games = games.merge(days_played_home, how='left', on=['date', 'detail_path'], sort=False)
#     games = games.merge(days_played_away, how='left', on=['date', 'detail_path'], sort=False)
#     return games

#####
##### **WIN-LOSS PERFORMANCE AND STREAKS IN LAST N GAMES**

In [13]:
def wins_n_games(games):
    teams_unique = list(set(get_team_names(games, output='df')['team']))+['Other']
    sequences = [3, 5, 10, 20, 50, 100]
    for j, team in enumerate(teams_unique):
        temp = games[games['team_pair'].str.contains(team)][['detail_path','home','away','team_pair','winner']]
        temp_home = games[games['home'].str.contains(team)][['detail_path','home','winner']]
        temp_away = games[games['away'].str.contains(team)][['detail_path','away','winner']]
        
        wins = np.array(np.where(temp['winner']==team, 1, 0))
        wins_home = np.array(np.where(temp_home['winner']==team, 1, 0))
        wins_away = np.array(np.where(temp_away['winner']==team, 1, 0))
        
        # streak test
        streak = [0,wins[0]]
        for i in range(1, len(wins)-1):
            next_val = streak[-1]+1 if wins[i]==1 else 0
            streak.append(next_val)
        temp['streak'] = np.asarray(streak)
        temp['streak_home'] = np.where(temp['home']==team, temp['streak'], -1)
        temp['streak_away'] = np.where(temp['away']==team, temp['streak'], -1)

        # streak home test
        streak_home = [0,wins_home[0]]
        for i in range(1, len(wins_home)-1):
            next_val = streak_home[-1]+1 if wins_home[i]==1 else 0
            streak_home.append(next_val)
        temp_home['streak_home_home'] = np.asarray(streak_home)
        
        # streak away test
        streak_away = [0,wins_away[0]]
        for i in range(1, len(wins_away)-1):
            next_val = streak_away[-1]+1 if wins_away[i]==1 else 0
            streak_away.append(next_val)
        temp_away['streak_away_away'] = np.asarray(streak_away)
        
        temp = temp.merge(temp_home[['detail_path','streak_home_home']], how='left', on='detail_path', sort=False)
        temp = temp.merge(temp_away[['detail_path','streak_away_away']], how='left', on='detail_path', sort=False)
        
        for s in sequences:
            temp['wins'+str(s)] = np.asarray([sum(wins[max(0,i-s):i]) for i in range(len(wins))])
            temp['wins'+str(s)+'_home'] = np.where(temp['home']==team, temp['wins'+str(s)], -1)
            temp['wins'+str(s)+'_away'] = np.where(temp['away']==team, temp['wins'+str(s)], -1)
        wins_df = temp if j==0 else pd.concat([wins_df, temp], axis=0)

        # return wins_df
    
    wins_df = wins_df[~wins_df['team_pair'].str.contains('Other')]

    home_streaks = wins_df[wins_df['streak_home']>-1].sort_index()
    away_streaks = wins_df[wins_df['streak_away']>-1].sort_index()

    home_wins = wins_df[wins_df['wins3_home']>-1].sort_index()
    away_wins = wins_df[wins_df['wins3_away']>-1].sort_index()
    
    wins_df = home_wins[['detail_path']]
    wins_df = wins_df.merge(home_streaks[['detail_path', 'streak_home', 'streak_home_home']], how='left', on='detail_path', sort=False)
    wins_df = wins_df.merge(away_streaks[['detail_path', 'streak_away', 'streak_away_away']], how='left', on='detail_path', sort=False)
    wins_df = wins_df.merge(home_wins[['detail_path']+['wins'+str(s)+'_home' for s in sequences]], how='left', on='detail_path', sort=False)
    wins_df = wins_df.merge(away_wins[['detail_path']+['wins'+str(s)+'_away' for s in sequences]], how='left', on='detail_path', sort=False)
    for col in list(wins_df.columns)[1:]:
        wins_df[col] = wins_df[col].fillna(0).astype(int)

    games = games.merge(wins_df, how='inner', on=['detail_path'], sort=-False).reset_index(drop=True)
    return games

#####
##### **WIN-LOSS PERFORMANCE AGAINST OPPONENTS**

In [14]:
def opponents(games):
    games['team_pair_sorted'] = np.asarray([g[:3]+'-'+g[-3:] if g[:3]<=g[-3:] else g[-3:]+'-'+g[:3] for g in games['team_pair']])
    unique_pairs = np.unique(games['team_pair_sorted'])

    for j, teams in enumerate(unique_pairs):
        temp = games[games['team_pair_sorted']==teams][['detail_path','team_pair_sorted','winner']]
        wins = np.array(np.where(temp['winner']==teams[:3], 1, 0))

        streak1, streak2 = [0], [0]
        for i in range(len(wins)-1):
            next_val1 = streak1[-1]+1 if wins[i]==1 else 0
            streak1.append(next_val1)
            next_val2 = streak2[-1]+1 if wins[i]==0 else 0
            streak2.append(next_val2)
        temp['streak1'] = np.asarray(streak1)
        temp['streak2'] = np.asarray(streak2)

        opponents_df = temp if j==0 else pd.concat([opponents_df, temp], axis=0)

    games = games.merge(opponents_df[['detail_path','team_pair_sorted','streak1','streak2']], how='left', on=['detail_path', 'team_pair_sorted'], sort=False)
    games['streak_opponents_home'] = games.apply(lambda r: r['streak1'] if r['home'] == r['team_pair_sorted'][:3] else r['streak2'], axis=1)
    games['streak_opponents_away'] = games.apply(lambda r: r['streak1'] if r['away'] == r['team_pair_sorted'][:3] else r['streak2'], axis=1)

    del games['streak1']
    del games['streak2']

    return games

#####
##### **GAMES IN LAST N DAYS**

In [15]:
def last_n_days(games):
    teams_unique = list(set(get_team_names(games, output='df')['team'])) #+['Other']
    sequences = [2, 3, 5, 10, 20]

    min_date = min(games['date'])
    max_date = max(games['date'])
    dates = [min_date + datetime.timedelta(days=d) for d in range((max_date - min_date).days)]

    for j, team in enumerate(teams_unique):
        temp = games[games['team_pair'].str.contains(team)][['date','detail_path','home','away']]
        d = np.array(temp['date'])
        temp['days_last_played'] = [0]+[(d[i]-d[i-1]).days for i in range(1,len(d))]

        y_dates = np.array(temp['date'])
        y_df = pd.DataFrame(y_dates,columns=['date'])
        y_df['game'] = 1
        n_dates = list(set(dates) - set(y_dates))
        n_df = pd.DataFrame(n_dates,columns=['date'])
        n_df['game'] = 0
        df = pd.concat([y_df, n_df], axis=0).sort_values(by='date').reset_index(drop=True)

        for s in sequences:
            game_seq = np.array(df['game'])
            cum_games = np.asarray([0]+[sum(game_seq[max(0,i-s):i]) for i in range(1,len(game_seq))])
            df['cum_games_'+str(s)] = cum_games

        temp_home = temp[temp['home']==team]
        temp_home = temp_home.merge(df, how='inner', on=['date'], sort=False)
        temp_home_all = temp_home if j==0 else pd.concat([temp_home_all, temp_home], axis=0)

        temp_away = temp[temp['away']==team]
        temp_away = temp_away.merge(df, how='inner', on=['date'], sort=False)
        temp_away_all = temp_away if j==0 else pd.concat([temp_away_all, temp_away], axis=0)

    temp_home_all = temp_home_all[['date','detail_path']+['cum_games_'+str(s) for s in sequences]+['days_last_played']]
    temp_home_all.columns = ['date','detail_path']+[c+'_home' for c in list(temp_home_all.columns)[2:]]

    temp_away_all = temp_away_all[['date','detail_path']+['cum_games_'+str(s) for s in sequences]+['days_last_played']]
    temp_away_all.columns = ['date','detail_path']+[c+'_away' for c in list(temp_away_all.columns)[2:]]        

    games = games.merge(temp_home_all, how='left', on=['date','detail_path'], sort=False)
    games = games.merge(temp_away_all, how='left', on=['date','detail_path'], sort=False)

    return games

#####
##### **SEASON STANDINGS**

In [40]:
def generate_standings(table, date, conf=None):

    if type(date) == str:
        date = datetime.datetime.strptime(date,'%Y-%m-%d').date()
    if type(table['date'][0]) == str:
        table['date'] = table['date'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').date())

    season = max(table[table['date']<date]['season'])
    games_season = table[(table['season']==season) & (table['date']<date)]
    games_season.loc[:,'count'] = 1
    games_preseason = games_season[games_season['game_type']=='preseason'].reset_index(drop=True)
    games_regular = games_season[games_season['game_type']=='regular'].reset_index(drop=True)
    games_playoffs = games_season[games_season['game_type']=='playoffs'].reset_index(drop=True)

    hw = games_regular[games_regular['home']==games_regular['winner']][['home','count','hs']].groupby('home').sum()
    hl = games_regular[games_regular['home']!=games_regular['winner']][['home','count']].groupby('home').sum()
    aw = games_regular[games_regular['away']==games_regular['winner']][['away','count','as']].groupby('away').sum()
    al = games_regular[games_regular['away']!=games_regular['winner']][['away','count']].groupby('away').sum()

    final_table = pd.concat([hw, hl, aw, al], axis=1).reset_index().fillna(0)

    final_table.columns = ['team', 'wins_home', 'points_home', 'losses_home', 'wins_away', 'points_away', 'losses_away']

    final_table['wins'] = final_table['wins_home']+final_table['wins_away']
    final_table['losses'] = final_table['losses_home']+final_table['losses_away']
    final_table['played'] = final_table['wins']+final_table['losses']
    final_table['diff'] = final_table['wins'] - final_table['losses']
    final_table['conf'] = final_table['team'].map(get_conference)

    final_table = final_table[['team', 'conf', 'played', 'wins', 'losses', 'wins_home', 'losses_home', 'points_home', 'wins_away', 'losses_away', 'points_away', 'diff']]
    final_table = final_table.sort_values(by=['diff', 'played'],ascending=[False, True]).reset_index(drop=True)

    if conf is not None:
        final_table = final_table[final_table['conf']==conf].reset_index(drop=True)

    final_table = final_table.reset_index()
    final_table.columns = ['pos']+list(final_table.columns)[1:]
    final_table['pos'] += 1

    return final_table


def build_standings(games):
    for i, d in enumerate(set(games['date'])):
        if i%500==0: print(i)
        try:
            standings = generate_standings(games, d)
            standings['date'] = d
            if i == 0:
                standings_all = standings
            else:
                standings_all = pd.concat([standings_all, standings], axis=0)
        except:
            print(d)
            pass
    standings_all = standings_all.sort_values(by=['date', 'pos'], ascending=[True, True])
    standings_all.to_csv('../data/processed/standings.csv', index=False)
    return standings_all


def standings(games):
    standing = pd.read_csv('../data/processed/standings.csv') # build_standings(games)
    try:
        standing['date'] = standing['date'].apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
    except:
        pass
    standings_home = standing[['team','date','wins','losses','wins_home','losses_home','points_home','wins_away','losses_away','points_away','diff']]
    standings_home.columns = ['team','date']+['home_standing_'+c for c in list(standings_home.columns)[2:]]
    standings_away = standing[['team','date','wins','losses','wins_home','losses_home','points_home','wins_away','losses_away','points_away','diff']]
    standings_away.columns = ['team','date']+['away_standing_'+c for c in list(standings_away.columns)[2:]]

    games = games.merge(standings_home, how='left', left_on=['home','date'], right_on=['team','date'], sort=False)
    games = games.merge(standings_away, how='left', left_on=['away','date'], right_on=['team','date'], sort=False)

    del games['team_x']
    del games['team_y']

    for c in list(games.columns)[-18:]:
        games[c] = games[c].fillna(0)

    games['standing_diff'] = games['home_standing_diff'] - games['away_standing_diff']
    games['standing_points_diff'] = games['home_standing_points_home'] - games['away_standing_points_away']

    return games

#####
##### **APPLY FEATURE TRANSFORMS SEQUENTIALLY**

In [70]:
games_final = etl_process(games, [basic_features, playoff_features, wins_n_games, opponents, last_n_days, standings])

In [44]:
games_final.iloc[-150:-100,:]

,date,away,as,home,hs,ot,detail_path,game_type,game_subtype,diff,...,away_standing_losses,away_standing_wins_home,away_standing_losses_home,away_standing_points_home,away_standing_wins_away,away_standing_losses_away,away_standing_points_away,away_standing_diff,standing_diff,standing_points_diff
36189,2023-02-14,SAC,109,PHX,120,0,sac-vs-phx-0022200870,regular,0,11,...,26.0,17.0,12.0,3216.0,13.0,14.0,1577.0,4.0,-2.0,675.0
36190,2023-02-14,WAS,126,POR,101,0,was-vs-por-0022200872,regular,0,25,...,30.0,14.0,12.0,2639.0,12.0,18.0,1409.0,-4.0,1.0,1651.0
36191,2023-02-15,CHI,113,IND,117,0,chi-vs-ind-0022200874,regular,0,4,...,30.0,17.0,11.0,4073.0,10.0,19.0,1177.0,-3.0,-6.0,870.0
36192,2023-02-15,CLE,112,PHI,118,0,cle-vs-phi-0022200875,regular,0,6,...,23.0,25.0,6.0,6867.0,12.0,17.0,1362.0,14.0,4.0,4260.0
36193,2023-02-15,DAL,109,DEN,118,0,dal-vs-den-0022200881,regular,0,9,...,28.0,20.0,9.0,4359.0,11.0,19.0,1264.0,3.0,17.0,2953.0
36194,2023-02-15,DET,109,BOS,127,0,det-vs-bos-0022200877,regular,0,18,...,43.0,9.0,20.0,3127.0,6.0,23.0,719.0,-28.0,56.0,7505.0
36195,2023-02-15,HOU,96,OKC,133,0,hou-vs-okc-0022200880,regular,0,37,...,44.0,8.0,20.0,931.0,5.0,24.0,607.0,-31.0,31.0,4617.0
36196,2023-02-15,MIA,105,BKN,116,0,mia-vs-bkn-0022200878,regular,0,11,...,28.0,19.0,10.0,3115.0,11.0,18.0,1208.0,2.0,9.0,1954.0
36197,2023-02-15,NOP,102,LAL,120,0,nop-vs-lal-0022200882,regular,0,18,...,28.0,21.0,9.0,4532.0,9.0,19.0,1078.0,2.0,-8.0,2638.0
36198,2023-02-15,NYK,122,ATL,101,0,nyk-vs-atl-0022200876,regular,0,21,...,25.0,20.0,11.0,7357.0,14.0,14.0,1614.0,9.0,-9.0,3192.0


In [43]:
games_final.to_csv('../data/processed/games_final.csv', index=False)
with open('../data/processed/games_final.pickle', 'wb') as f: pickle.dump(games_final, f, pickle.HIGHEST_PROTOCOL)

##### **MORE FEATURES TO COME!**

In [44]:
# number of players used in season
# number of players used in last N games
# rank of player in season
# rank of player in last N games
# travel factors (climate, distance, timezones)
# average score overall
# average score in last N games
# average winning margin

# average winning margin weighted by opponent placement
# player-by-player battles
# presence of individual players
# avg age on court
# max age on court
# avg height of players
# avg weight of players
# long-term and short-term stats per player

# roster stability coefficient (we can come up with something ;))